In [50]:
import numpy as np

In [51]:
from scipy.io import loadmat

train = loadmat('C:\\Users\\gencee\\Dropbox\\eth\\master\\MT\\TIDIGITS_task\\data\\tidigits_mfccs_train.mat')['tidigits_mfccs_train']
test = loadmat('C:\\Users\\gencee\\Dropbox\\eth\\master\\MT\\TIDIGITS_task\\data\\tidigits_mfccs_test.mat')['tidigits_mfccs_test']


In [52]:
sampTr = train['mfccs_third'][0][0][0] 
labTr = train['idx_labels'][0][0]
sampTe = test['mfccs_third'][0][0][0]
labTe = test['idx_labels'][0][0]

In [53]:
#find max length sample to use it as reference for padding
maxlenTr= 0
maxlenTe= 0
for i in range(sampTr.shape[0]):
    if sampTr[i].shape[1]>maxlenTr:
        maxlenTr = sampTr[i].shape[1]
        indexTr = i
        
for i in range(sampTe.shape[0]):
    if sampTe[i].shape[1]>maxlenTe:
        maxlenTe = sampTe[i].shape[1]
        indexTe = i
        
if maxlenTe>maxlenTr:
    maxl=maxlenTe
else:
    maxl=maxlenTr
    

In [54]:
## REMARK 1 - padding
# If not all the sequence have the same length you HAVE TO pre-pad them.
# The reason is simple, when you feed them in the RNN you then take the last activation to do the classification, but
# if the last activation comes after 100 or more steps of zeros the input will be washed out and also the gradient will not flow.

#pad out all training and test samples
trainObs=np.zeros([sampTr.shape[0],sampTr[0].shape[0],maxl])
testObs=np.zeros([sampTe.shape[0],sampTe[0].shape[0],maxl])

for i in range(sampTr.shape[0]):
    s = sampTr[i].shape
    trainObs[i] = np.concatenate((np.zeros((39, maxl - s[1])), sampTr[i]), axis=1).T  ## you should pad with zeros in the beginning
#     for j in range(sampTr[i].shape[0]):
#         trainObs[i,j,:]= np.lib.pad(sampTr[i][j],(0,maxl-s[1]),'constant',constant_values=(0,0))
        
for i in range(sampTe.shape[0]):
    s = sampTe[i].shape
    testObs[i] = np.concatenate((np.zeros((39, maxl - s[1])), sampTe[i]), axis=1).T  ## pad with zeros in the beginning
#     for j in range(sampTe[i].shape[0]):
#         testObs[i,j,:]= np.lib.pad(sampTe[i][j],(0,maxl-s[1]),'constant',constant_values=(0,0))
              
    

In [55]:
#convert class labels from scalars to one-hot vectors
def dense_to_one_hot(labels_dense, num_classes=10):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [56]:
labelsTr = dense_to_one_hot(labTr[0],num_classes=11)
labelsTe = dense_to_one_hot(labTe[0],num_classes=11)

In [57]:
#shuffle data
p = np.random.permutation(trainObs.shape[0])
a = np.zeros_like(trainObs)
a = trainObs[p,:,:]
b = np.zeros_like(labelsTr)
b = labelsTr[p,:]
trainObs = a
labelsTr = b

p = np.random.permutation(testObs.shape[0])
a = np.zeros_like(testObs)
a = testObs[p,:,:]
b = np.zeros_like(labelsTe)
b = labelsTe[p,:]
testObs = a
labelsTe = b

In [58]:
import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell, MultiRNNCell,BasicLSTMCell
from tensorflow.python.ops import rnn

In [59]:
tf.reset_default_graph()

learning_rate = 0.001
training_iters = 120
batch_size = 100
display_step = 40

n_input = 39
n_steps = maxl
n_hidden = 128
n_classes = 11

## REMARK 2 - sequence
# Unless you tell tensorflow, the order for the tesnrof dimension is 
# [batch_size, time_steps, features]
# Here is the main mistake you made you where running the RNN alogn the features dimension instead of the time dimension.
# Should be :
# x = tf.placeholder('float',[None, n_steps, n_input])
# y = tf.placeholder('float',[None, n_classes])

x = tf.placeholder('float',[None, n_input, n_steps])
y = tf.placeholder('float',[None, n_classes])

In [60]:
def RNN(x):
    layer = {'weights':tf.Variable(tf.random_normal([n_hidden,n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}
    cell = LSTMCell(n_hidden,state_is_tuple=True)
    cell = MultiRNNCell([cell] * 2)
    output, states = rnn.dynamic_rnn(cell, x, dtype = tf.float32)
    
    # Very good !
    output = tf.transpose(output, [1, 0, 2])
    last = tf.gather(output, int(output.get_shape()[0]) - 1)

    output = tf.matmul(last,layer['weights']) + layer['biases']
    
    return tf.nn.softmax(output) 

In [61]:
prediction =RNN(x)

## REMARK 3 - cost function
# Even though the formulation is correct, the calculation you make here is dangerous.
# What happens if your activation is 0? 
# Here you can do 2 things, either you correct the numerical problems by adding a small value to the log (e.g. 1e-10)
# or you use tensorflow built-in cost function that takes care of that for you.
# 
# cost= tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y)
# where logits is the simple output of the network, the softmax is done inside the function.

# define cost and optimizer
cost= -tf.reduce_sum(y* tf.log(prediction))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
#evaluate model
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, 'float'))


C:\Users\gencee\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [62]:
with tf.Session() as session:
    session.run(tf.initialize_all_variables())
    
    for itr in range(training_iters):
        offset = (itr * batch_size) % (labelsTr.shape[0] - batch_size)
        batch_x = trainObs[offset:(offset + batch_size),:,:]
        batch_y = labelsTr[offset:(offset + batch_size), :]
        _, c = session.run([optimizer, cost],feed_dict={x: batch_x, y : batch_y})
           
        if itr % display_step == 0:
            # Calculate batch accuracy
            acc = session.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(itr) + ", Training Accuracy= " + "{:.5f}".format(acc))
    acc = session.run(accuracy, feed_dict={x: testObs, y: labelsTe})
    print("Test Accuracy= " + "{:.5f}".format(acc))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iter 0, Training Accuracy= 0.29000
Iter 40, Training Accuracy= 0.41000
Iter 80, Training Accuracy= 0.96000
Test Accuracy= 0.24497
